# Assembling or loading anndata object
**In case assemble new object, it will be used in the velocity analysis**

# A: set up

In [1]:
#####################DEFINING TEST NUMBER##############################
test="Test1"

##########################DEFINING IMPUTS##############################
path_solo_quant="/mnt/workspace/gvalent/5_SC_automatation/data1/raw_data" #This is the directory where the quant folder from snakemake preprocessing is located
path_outout="/mnt/workspace/gvalent/5_SC_automatation/data1/raw_data" #The path were the folders results/Test* will be created to export your results

#To create the anndata object for velocity analyis, paste bellow the sample descriptions of 10X.yml
the_10X_yml=[    
"sample1:condition:room_air",
"sample2:condition:room_air",
"sample3:condition:smooke_exposure",
"sample4:condition:smooke_exposure",
]

#Which Solo dataset do you wanna use?
data_type='filtered' #set 'raw' or 'filtered'

# B: checking and creating output directory

In [2]:
import checker
import creators
from imp import reload
reload(checker)
reload(creators)

checker.input_path_velocity(path_solo_quant, the_10X_yml, data_type) #Check if the input path exist and also if the necessary data to assembling the anndata for velocity is there
checker.output_path(path_outout, test) #Check if the output path exist. Otherwise, it will create the directory

The info.yml was created.
Sample sample1 have the proper data for assembling an anndata for velocity analysis.
Sample sample2 have the proper data for assembling an anndata for velocity analysis.
Sample sample3 have the proper data for assembling an anndata for velocity analysis.
Sample sample4 have the proper data for assembling an anndata for velocity analysis.
Creating the /mnt/workspace/gvalent/5_SC_automatation/data1/raw_data/results directory.
The /mnt/workspace/gvalent/5_SC_automatation/data1/raw_data/results directory is ready.
Creating the /mnt/workspace/gvalent/5_SC_automatation/data1/raw_data/results/Test1 directory.
The /mnt/workspace/gvalent/5_SC_automatation/data1/raw_data/results/Test1 directory is ready.
The info.yml has a new key.


# C: assembling anndata

In [3]:
import assemblers
import checker
from imp import reload
reload(assemblers)
reload(checker)

adata=assemblers.velocity(the_10X_yml, test, data_type) #This module will assembling the 10X anndata object able to run velocity analysis.
display(adata)

Runing sample number 1 out of 4 samples.
		Loading matrix to compose the X object: sample1
		Loading genes and cells identifiers to make the obs object: sample1
		Loading the gene features to make the var object: sample1
		Loading spliced, unspliced and ambigous matrix to compose the X object: sample1


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


		Creating partial anndata object: sample1
Runing sample number 2 out of 4 samples.
		Loading matrix to compose the X object: sample2
		Loading genes and cells identifiers to make the obs object: sample2
		Loading the gene features to make the var object: sample2
		Loading spliced, unspliced and ambigous matrix to compose the X object: sample2


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


		Creating partial anndata object: sample2
Runing sample number 3 out of 4 samples.
		Loading matrix to compose the X object: sample3
		Loading genes and cells identifiers to make the obs object: sample3
		Loading the gene features to make the var object: sample3
		Loading spliced, unspliced and ambigous matrix to compose the X object: sample3


...

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


		Creating partial anndata object: sample4
Creating the final anndata object.
		Concatenating objects.


... storing 'condition' as categorical
... storing 'feature_types' as categorical


		Renaming batches.
		Saving and loading.
Loading the anndata for velocity and storing as an adata variable.


AnnData object with n_obs × n_vars = 40377 × 55359
    obs: 'condition'
    var: 'gene_ids', 'feature_types'
    layers: 'ambiguous', 'spliced', 'unspliced'